# Parallel computing

### Start and check multiple processes

In [1]:
using BenchmarkTools

In [2]:
addprocs(2) # or start julia in a console with the option -p n_processes
# this start multiple processes not threads (processes doesn't share memory as threads does, but threads
# are limited to a single CPU)

2-element Array{Int64,1}:
 2
 3

In [3]:
for pid in workers()
    println(pid)
end
    

2
3


In [4]:
@everywhere println(myid())

1
	From worker 2:	2
	From worker 3:	3


### Parallel for -  @parallel for macro
- output_var = @parallel (aggregator) for_cycle 

In [5]:
function f(n)
  s = 0.0
  for i = 1:n
    s += i/2
  end
    return s
end

f (generic function with 1 method)

In [6]:
@time f(100000000)

  0.088311 seconds (1.64 k allocations: 76.989 KB)


2.500000025e15

In [7]:
function pf(n)
    s = @parallel (+) for i = 1:n # aggregate using sum on variable s 
        i/2                       # last element of for cycle is used by the aggregator
  end
  return s
end

pf (generic function with 1 method)

In [8]:
@time pf(100000000)

  0.524146 seconds (388.17 k allocations: 16.727 MB)


2.500000025e15

### Parallel map - pmap(f,collection)
 - applies a function f on each elements of the collection

In [9]:
x = [rand(100,100) for i in 1:10];

In [10]:
@benchmark map(svd,x)

BenchmarkTools.Trial: 
  memory estimate:  5.47 MiB
  allocs estimate:  242
  --------------
  minimum time:     26.510 ms (0.00% GC)
  median time:      26.827 ms (0.00% GC)
  mean time:        26.918 ms (0.53% GC)
  maximum time:     28.537 ms (0.00% GC)
  --------------
  samples:          186
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [11]:
@benchmark pmap(svd,x)

BenchmarkTools.Trial: 
  memory estimate:  1.62 MiB
  allocs estimate:  1858
  --------------
  minimum time:     15.174 ms (0.00% GC)
  median time:      15.903 ms (0.00% GC)
  mean time:        16.020 ms (0.53% GC)
  maximum time:     25.461 ms (0.00% GC)
  --------------
  samples:          312
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%